In [7]:
import requests
import json
import re
from bs4 import BeautifulSoup

ama_url = str(input('Enter amazon.in reviews link here '))
product_id = ama_url.rsplit("reviews//")[1].rsplit('/')[0]
f = open("output.txt", "a")
review_url = 'https://www.amazon.in/hz/reviews-render/ajax/reviews/get/' 
start_url = 'https://www.amazon.in/product-reviews/{}/'.format(product_id)

session = requests.Session()
session.headers.update({
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36'
    })

session.get(start_url)

def parse_reviews(reply):
    reviews = []
    for fragment in reply.split('&&&'):
        if not fragment.strip():
            continue
        json_fragment = json.loads(fragment)
        if json_fragment[0] != 'append':
            continue
        html_soup = BeautifulSoup(json_fragment[2], 'html.parser')
        div = html_soup.find('div', class_='review')
        if not div:
            continue
        review_id = div.get('id')
        if not review_id:
            continue
        review_classes = ' '.join(html_soup.find(class_='review-rating').get('class'))
        rating = re.search(r'a-star-(\d+)', review_classes).group(1)
        title = html_soup.find(class_='review-title').get_text(strip=True)
        review = html_soup.find(class_='review-text').get_text(strip=True)
        reviews.append({'review_id': review_id,
                        'rating': rating,
                        'title': title,
                        'review': review})
    return reviews

def get_reviews(product_id, page):
    data = {
        'sortBy':'',
        'reviewerType':'all_reviews',
        'formatType':'',
        'mediaType':'',
        'filterByStar':'all_stars',
        'pageNumber':page,
        'filterByKeyword':'',
        'shouldAppend':'undefined',
        'deviceType':'desktop',
        'reftag':'cm_cr_getr_d_paging_btm_{}'.format(page),
        'pageSize':10,
        'asin':product_id,
        'scope':'reviewsAjax0'
        }
    r = session.post(review_url + 'ref=' + data['reftag'], data=data)
    reviews = parse_reviews(r.text)
    return reviews

page = 1
while True:
    print('Scraping page', page)
    reviews = get_reviews(product_id, page)
    if not reviews:
        break
    for review in reviews:
        print(' | ', review['rating'],' | ', review['title'],' | ')
        f.write('|{}| {}|\n'.format(review['rating'], review['title']))
    page += 1
f.close()

Enter amazon.in reviews link here https://www.amazon.in/product-reviews//B08L5V9T31/ref=acr_dp_hist_5?ie=UTF8&filterByStar=five_star&reviewerType=all_reviews#reviews-filter-bar
Scraping page 1
 |  1  |  Worst phone from an irresponsible company  | 
 |  4  |  A perfect phone without needed accessories.  | 
 |  5  |  Apple's latest 'New-Gen' marvel!!  | 
 |  1  |  Received fake mobile  | 
 |  5  |  iPhone 12 Pro  | 
 |  5  |  Camera is what makes this phone worth every single penny spent on it  | 
 |  5  |  Very good  | 
 |  1  |  Device not working, defective piece.  | 
 |  5  |  In love with the 12 pro.  | 
 |  1  |  Completely a duplicate copy of original as it has all android os  | 
Scraping page 2
 |  5  |  Awesome camera  | 
 |  4  |  Please dont buy who lives in High altitude  | 
 |  5  |  Perfect and classic  | 
 |  1  |  Good but  | 
 |  4  |  Built powerful yet underwhelming  | 
 |  5  |  My First IPhone Is Best...😀👍  | 
 |  1  |  Check b4 buy  | 
 |  5  |  A great phone  | 
 |